In [ ]:
import pandas as pd
import os
import ndjson
import time
import requests
from bs4 import BeautifulSoup as soupify

In [ ]:
# function to test the link
def test_link(link):
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"} 
    
    # link 
    r = requests.get(link, headers=headers, verify=False)
    content = soupify(r.text, "html.parser") 
    if content.find(id = "ContentPlaceHolder1_HyperLink1") or content.find(id = "ContentPlaceHolder1_HyperLink2") or content.find(id = "ContentPlaceHolder1_lnkPdfDownload"):
        status = "OK"
    else:
        status = "Link Not on page"
    return status

In [ ]:
tar_folder = os.path.join('data')
tar_files = [i for i in os.listdir(tar_folder) if(i.endswith("ndjson"))]

In [ ]:
all_df = pd.DataFrame()

for tar_file in tar_files:
    tmp_df = pd.read_json(os.path.join(tar_folder, tar_file), lines = True)
    tmp_df = tmp_df.dropna(subset = ['Q_No'], axis = 'index')
    tmp_df['src_file'] = tar_file
    all_df = pd.concat([all_df, tmp_df])

In [ ]:
# update the link 
all_df['New_Link'] = all_df['Q_Link'].apply(lambda x: x.replace('http://164.100.47.194/Loksabha/', 'https://loksabha.nic.in/'))

In [ ]:
new_df = all_df.copy()
new_df["Status"] = ""

In [ ]:
# test updated link
# save status
test_index = range(1, len(new_df), 3000)
for index in test_index:
    temp_link = new_df.iloc[index]['Q_Link']
    link_status = test_link(temp_link)
    new_df.loc[index, ['Status']] = link_status
    print(link_status)
    time.sleep(0.5)

In [ ]:
new_df['Status'].value_counts()